In [1]:
import pandas as pd
import h5py
import numpy as np
import matplotlib.pyplot as plt
import os
import pickle
import pymetis
import networkx as nx
import time
from networkx.algorithms import community
from random import shuffle
import math
import torch
import torch.nn as nn
import torch_geometric as tg
import torch.nn.functional as F
from torch_geometric.nn import MessagePassing
from torch_geometric.nn import GCNConv
from torch_geometric.utils import add_self_loops, degree
from torch.nn import init
import pdb
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from torch.utils.data import Dataset, DataLoader, random_split, ConcatDataset
from torch_geometric.data import Data
import torch.optim as optim
import pywt
from scipy.stats import norm
import scipy.interpolate as interp

In [2]:
def save_data(data, file_path):
    with open(file_path , 'wb') as f:
        pickle.dump(data,f)
        f.close()

In [3]:
def save_data_hdf5(data, file_path):
    with h5py.File(file_path, 'w') as f:
        for key, value in data.items():
            f.create_dataset(key, data=value)

In [4]:
def open_data(file_path):
    file = open(file_path,"rb")
    raw_data = pickle.load(file)  
    return raw_data

In [14]:
def wavelet_transform(data, level = 4):
    volume = data[:,:,:]
    time_steps = volume.shape[0]
    #volume shape:天数，点数，点特征
    trend_patterns = []
    periodic_patterns = []
    
    for i in range(volume.shape[1]):
        #对每一个节点的四个方向进行小波变换
        #pywt.wavedec输出是一个包含几个数组的列表 ,提取低频/逼近系数
        four_dir_coeffs = [pywt.wavedec(volume[:, i, j], wavelet= 'db1', level = level) for j in range(volume.shape[2])]
        #重构信号仅使用逼近系数
        freq_coeffs = []
        
        for each_dir_coeffs in four_dir_coeffs:
            approximation = each_dir_coeffs[0]
            details = each_dir_coeffs[1:] 
            coeffs_trend_only = [approximation] + [np.zeros_like(detail) for detail in details]
            freq_coeffs.append(pywt.waverec(coeffs_trend_only, wavelet= 'db1'))

        trend_patterns.append(np.transpose(np.array(freq_coeffs)))
        #重构去除趋势后的周期部分
        four_dir_period = [pywt.waverec([np.zeros_like(each_dir_coeffs[0])] + each_dir_coeffs[1:], wavelet= 'db1') for each_dir_coeffs in four_dir_coeffs]
        periodic_patterns.append(np.transpose(np.array(four_dir_period)))  
     
    #Shape: node_num, time_step, 4
    trend_patterns = np.array(trend_patterns)
    periodic_patterns = np.array(periodic_patterns) 
    
    
    #Change FROM node_num, time_step, 4 TO time step, node_num, 4
    return np.transpose(np.array(trend_patterns), (1,0,2)),  np.transpose(np.array(periodic_patterns), (1,0,2))

In [15]:
def create_sequences(data, seq_len, pred_len):
    sequences = []
    for i in range(data.shape[0] - seq_len - pred_len + 1):
        input_seq = data[i:i + seq_len, :, :]
        target_seq = data[i + seq_len:i + seq_len + pred_len, :, :]
        sequences.append((input_seq, target_seq))
    return sequences


def sequence_concatenate(sequences):
    for i in range(len(sequences)):
        input_seq, target_seq = sequences[i]
        input_seq = np.expand_dims(input_seq, axis=0)
        target_seq = np.expand_dims(target_seq, axis=0)
        if i == 0:
            input_result = input_seq
            target_result = target_seq
        else:
            input_result = np.concatenate((input_result, input_seq), axis=0)
            target_result = np.concatenate((target_result, target_seq), axis=0)
    return input_result, target_result

In [16]:
def input_target_divide(city_name, file_root_path, seq_len, pred_len):
    scaler = MinMaxScaler()
    file_path = file_root_path + city_name + "/volume_data/volume_file.h5"
    data = open_data(file_path)
    city_node_num = data.shape[1]
    #----------只取横跨疫情前后100天的数据（从2020年2月1日起的100天）-----------------------------------
    data = data[181+30:181+30+100, :, :]
    day_num, node_num, feature_num = data.shape[0], data.shape[1], data.shape[2]-1
    #---------归一化数据-----------------
    volume_data = data[ :, :, 1:]
    nodes_id = np.expand_dims(data[:,:,0], axis= -1)
    
    norm_volume_data = scaler.fit_transform(np.reshape(volume_data, (day_num * node_num, feature_num)))
    norm_volume_data = np.reshape(norm_volume_data, (day_num, node_num, feature_num))
    norm_volume_data = np.concatenate((nodes_id, norm_volume_data), axis = -1)
    
    sequences = create_sequences(norm_volume_data, seq_len, pred_len)
    input_result, target_result = sequence_concatenate(sequences)
    
    #Input shape: 100//seq , seq, Node_num, [Node_ID+ Node_Feature_num(5)]
    return input_result, target_result, city_node_num
    
    

In [17]:
def Wavelet_Transform(city_name, input_result, target_result, max_len):
    target_template = np.zeros((target_result.shape[0], target_result.shape[1], max_len, target_result.shape[3] + 1))
    
    trend_template = np.zeros((input_result.shape[0], input_result.shape[1], max_len, input_result.shape[3] + 1) )
    period_template = np.zeros((input_result.shape[0], input_result.shape[1], max_len, input_result.shape[3] + 1)) 
    for i in range(input_result.shape[0]):
        #----------Use Wavelet Transform to extract TREND AND PERIODIC FEATURES---
        trend_parts, periodic_parts = wavelet_transform(input_result[i, :, :, 1:], level = 4)
        #----------------------------------------------------------------------------
        trend_template[i, :, :input_result.shape[2], 1:5] = trend_parts
        trend_template[i, :, :input_result.shape[2], 0] = input_result[i, :, :, 0]
        
        
        #----------------------------------------------------------------------------
        period_template[i, :, :input_result.shape[2], 1:5] = periodic_parts
        period_template[i, :, :input_result.shape[2], 0] = input_result[i, :, :, 0]
        
        
        #------------------Target covers 4-Direction Volumes AND Target LABLE--------------------------
        target_template[i, :, :input_result.shape[2], 1:5] = target_result[i, :, :, 1:5]
        target_template[i, :, :input_result.shape[2], 0] = target_result[i, :, :, 0]
        
        if city_name == "Barcelona":
            trend_template[i, :, :input_result.shape[2], 5] = 1.0
            period_template[i, :, :input_result.shape[2], 5] = 1.0
            
            target_template[i, :, :input_result.shape[2], 5] = 1.0
            
            
        else:
            trend_template[i, :, :input_result.shape[2], 5] = 0.0
            period_template[i, :, :input_result.shape[2], 5] = 0.0
            
            target_template[i, :, :input_result.shape[2], 5] = 0.0
        
    return trend_template, period_template, target_template

In [18]:
seq_len, pred_len = 20, 10
city_names = ["Barcelona", "Antwerp"]
file_root_path = "D:ThesisData/processed data/"
max_len = 111260#The Node Num of Antwerp

input_result, target_result, city_node_num = input_target_divide(city_names[1], file_root_path, seq_len, pred_len)
trend_template, period_template, target_template = Wavelet_Transform(city_names[1], input_result, target_result, max_len)
print("One city finished")
city_dictionary = {"trend_template": trend_template, "period_template": period_template, "target_template": target_template, "city_node_num":city_node_num}
save_data_hdf5(city_dictionary, file_root_path + city_names[1] + "/input_target/input_target.h5")
    

One city finished
